In [64]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
movies = pd.read_csv('movies.csv')
movies

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [20]:
movies.iloc[:]['genres']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                         NaN
4802                                 Documentary
Name: genres, Length: 4803, dtype: object

In [11]:
len(movies)

4803

In [23]:
df_with_selected_columns = movies.iloc[:][['genres', 'keywords', 'cast', 'director']]
df_with_selected_columns

,genres,keywords,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [55]:
df_with_selected_columns = df_with_selected_columns.fillna('')

In [56]:
df_with_selected_columns.isna()

,genres,keywords,cast,director
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
4798,False,False,False,False
4799,False,False,False,False
4800,False,False,False,False
4801,False,False,False,False


In [57]:
joined_features = pd.Series('', index = range(len(df_with_selected_columns)))

for column in df_with_selected_columns.columns:
    joined_features = df_with_selected_columns[column] + ' ' + joined_features
joined_features

0       James Cameron Sam Worthington Zoe Saldana Sigo...
1       Gore Verbinski Johnny Depp Orlando Bloom Keira...
2       Sam Mendes Daniel Craig Christoph Waltz L\u00e...
3       Christopher Nolan Christian Bale Michael Caine...
4       Andrew Stanton Taylor Kitsch Lynn Collins Sama...
                              ...                        
4798    Robert Rodriguez Carlos Gallardo Jaime de Hoyo...
4799    Edward Burns Edward Burns Kerry Bish\u00e9 Mar...
4800    Scott Smith Eric Mabius Kristin Booth Crystal ...
4801    Daniel Hsia Daniel Henney Eliza Coupe Bill Pax...
4802    Brian Herzlinger Drew Barrymore Brian Herzling...
Length: 4803, dtype: object

In [66]:
tfidfVectorizer = TfidfVectorizer()
tfidfValues = tfidfVectorizer.fit_transform(joined_features)
type(tfidfValues)

scipy.sparse._csr.csr_matrix

In [75]:
similarity_matrix = pd.DataFrame(cosine_similarity(tfidfValues))
similarity_matrix

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.027039,0.044791,0.007896,0.114058,0.071865,0.0,0.041171,0.024869,0.030587,...,0.000000,0.000000,0.000000,0.020007,0.000000,0.041200,0.000000,0.000000,0.00000,0.000000
1,0.027039,1.000000,0.015052,0.006808,0.014699,0.114475,0.0,0.014261,0.036782,0.026371,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005007,0.000000,0.016538,0.00000,0.000000
2,0.044791,0.015052,1.000000,0.062943,0.075407,0.044188,0.0,0.088494,0.041202,0.051567,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.014666,0.000000,0.000000,0.06576,0.000000
3,0.007896,0.006808,0.062943,1.000000,0.007961,0.043877,0.0,0.007723,0.036850,0.112528,...,0.015547,0.008505,0.028597,0.038129,0.013734,0.030554,0.000000,0.003731,0.00000,0.000000
4,0.114058,0.014699,0.075407,0.007961,1.000000,0.151449,0.0,0.075335,0.010052,0.050359,...,0.000000,0.030153,0.000000,0.050553,0.000000,0.005855,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.041200,0.005007,0.014666,0.030554,0.005855,0.005730,0.0,0.018688,0.000000,0.005664,...,0.000000,0.006256,0.015159,0.004299,0.010101,1.000000,0.000000,0.000000,0.00000,0.000000
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.005761,0.000000,0.000000,0.000000,0.000000,1.000000,0.012605,0.00000,0.000000
4800,0.000000,0.016538,0.000000,0.003731,0.000000,0.018924,0.0,0.000000,0.000000,0.000000,...,0.002987,0.005371,0.003100,0.002718,0.000000,0.000000,0.012605,1.000000,0.00000,0.025548
4801,0.000000,0.000000,0.065760,0.000000,0.000000,0.000000,0.0,0.000000,0.065673,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000


In [89]:
movies_names_list = movies['original_title']
movies_names_list.name = ''
movies_names_list

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: , Length: 4803, dtype: object

In [82]:
similarity_matrix.index = movies_names_list
similarity_matrix.columns = movies_names_list
similarity_matrix

,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.027039,0.044791,0.007896,0.114058,0.071865,0.0,0.041171,0.024869,0.030587,...,0.000000,0.000000,0.000000,0.020007,0.000000,0.041200,0.000000,0.000000,0.00000,0.000000
Pirates of the Caribbean: At World's End,0.027039,1.000000,0.015052,0.006808,0.014699,0.114475,0.0,0.014261,0.036782,0.026371,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005007,0.000000,0.016538,0.00000,0.000000
Spectre,0.044791,0.015052,1.000000,0.062943,0.075407,0.044188,0.0,0.088494,0.041202,0.051567,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.014666,0.000000,0.000000,0.06576,0.000000
The Dark Knight Rises,0.007896,0.006808,0.062943,1.000000,0.007961,0.043877,0.0,0.007723,0.036850,0.112528,...,0.015547,0.008505,0.028597,0.038129,0.013734,0.030554,0.000000,0.003731,0.00000,0.000000
John Carter,0.114058,0.014699,0.075407,0.007961,1.000000,0.151449,0.0,0.075335,0.010052,0.050359,...,0.000000,0.030153,0.000000,0.050553,0.000000,0.005855,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
El Mariachi,0.041200,0.005007,0.014666,0.030554,0.005855,0.005730,0.0,0.018688,0.000000,0.005664,...,0.000000,0.006256,0.015159,0.004299,0.010101,1.000000,0.000000,0.000000,0.00000,0.000000
Newlyweds,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.005761,0.000000,0.000000,0.000000,0.000000,1.000000,0.012605,0.00000,0.000000
"Signed, Sealed, Delivered",0.000000,0.016538,0.000000,0.003731,0.000000,0.018924,0.0,0.000000,0.000000,0.000000,...,0.002987,0.005371,0.003100,0.002718,0.000000,0.000000,0.012605,1.000000,0.00000,0.025548


In [98]:
def returnTop10_SimilarMovies(similarity_mat, movie_name):
    cosine_values = similarity_mat[movie_name]
    sorted_similar_movies = cosine_values.sort_values(ascending = False)
    return sorted_similar_movies[1:11].index.tolist()
    
returnTop10_SimilarMovies(similarity_matrix, 'Harry Potter and the Half-Blood Prince')

['Harry Potter and the Goblet of Fire',
 'Harry Potter and the Order of the Phoenix',
 "Harry Potter and the Philosopher's Stone",
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Prisoner of Azkaban',
 'Wild Target',
 'Nanny McPhee',
 'Seventh Son',
 'Rise of the Planet of the Apes',
 'The Borrowers']